# Encryprtion/Decryption/Hashing

### What is RSA?
RSA is like having two special keys: one you can share with everyone (public key) and one that only you should have (private key). These keys are used to lock (encrypt) and unlock (decrypt) secret messages.

- Public Key: This is like a padlock you give to anyone. People can use it to lock a message for you.
- Private Key: This is like the key to the padlock. Only you have it, and you use it to unlock the messages sent to you.

### 1. Key Generation (Creating the Keys)
In the code, we're using a function called generate_keys() to make both keys:

- Public key: Used to lock the message.
- Private key: Used to unlock the message.

*public_key, private_key = generate_keys()*

After making the keys, the code converts them into a simple format (hexadecimal, which is like a simplified number format) so we can store or share them easily. That’s what serialize_private_key_to_hex() and serialize_public_key_to_hex() do.

#### --> AFTER YOU GENERATE A RANDOM KEY PAIR, PLEASE KEEP IT SAFE, PUBLIC KEY MUST BE PUBLISHD IN CANVAS, PRIVATE KEY MUST BE KEPT IN SECRET AND SAFETY STORED !!!! 

### 2. Encryption (Locking the Message)

To consider a key secure, key sizes are 2048 bits or longer. In this case, we are using a key size of 2048

Now, someone has your public key and they want to send you a secret message. They use this key to lock (encrypt) the message so no one else can read it. Here's the process in the code:

*encrypted_message = encrypt_message(plaintext_message, public_key)*

The person writes a message (like "Hello").
The message is "locked" using your public key.
The result is a weird-looking scrambled message that only you can unlock.
This step is important because even if someone intercepts the scrambled message, they can’t understand it without the private key.

### 3. Decryption (Unlocking the Message)
When you get the locked message, you use your private key to unlock (decrypt) it and read the original message.

*decrypted_message = decrypt_message(encrypted_message, private_key)*

You take the scrambled message.
Use your private key to unlock it.
Now, you can see the original message.


In [39]:
## Import all the necessary libraries for the code
import base64
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.exceptions import InvalidSignature
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend


# Define functions for the entire code

def generate_keys():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    public_key = private_key.public_key()
    return public_key, private_key

# Function to serialize private key and convert to hex

def serialize_private_key_to_hex(private_key):
    private_key_bytes = private_key.private_bytes(
        encoding=serialization.Encoding.DER,
        format=serialization.PrivateFormat.TraditionalOpenSSL,
        encryption_algorithm=serialization.NoEncryption()
    )
    return private_key_bytes.hex()

# Function to serialize public key and convert to hex

def serialize_public_key_to_hex(public_key):
    public_key_bytes = public_key.public_bytes(
        encoding=serialization.Encoding.DER,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    return public_key_bytes.hex()

# Function to deserialize hex back to private key

def deserialize_private_key_from_hex(hex_data):
    private_key_bytes = bytes.fromhex(hex_data)
    private_key = serialization.load_der_private_key(
        private_key_bytes,
        password=None
    )
    return private_key

# Function to deserialize hex back to public key

def deserialize_public_key_from_hex(hex_data):
    public_key_bytes = bytes.fromhex(hex_data)
    public_key = serialization.load_der_public_key(
        public_key_bytes
    )
    return public_key

# Digital signing 
def sign_message(message, private_key):
    signature = private_key.sign(
        message.encode('utf-8'),  # Encode the message to bytes
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    return base64.b64encode(signature).decode('utf-8')  # Return Base64-encoded signature

# Signature Verification
def verify_signature(message, signature, public_key):
    try:
        public_key.verify(
            base64.b64decode(signature),  # Decode Base64 to bytes
            message.encode('utf-8'),  # Message needs to be bytes
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except Exception as e:
        return False

# Hashing function
def get_text_hash(text: str) -> str:
    # Create a SHA-256 hash object
    digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
    
    # Update the hash object with the bytes of the text
    digest.update(text.encode('utf-8'))
    
    # Finalize the hash and get the digest
    hash_bytes = digest.finalize()
    
    # Convert the hash bytes to a hexadecimal string
    return hash_bytes.hex()

def encrypt_message(plaintext, public_key):
    ciphertext = public_key.encrypt(
        plaintext.encode('utf-8'),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return base64.b64encode(ciphertext).decode('utf-8')  # Convert to base64 string

def decrypt_message(ciphertext, private_key):
    decrypted = private_key.decrypt(
        base64.b64decode(ciphertext),  # Decode from base64 to bytes
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return decrypted.decode('utf-8')  # Convert decrypted bytes to string

In [33]:
# Block 1: Key Generation

public_key, private_key = generate_keys()
print("Keys generated successfully. \n")
print(f"Private key (hex format): \n{serialize_private_key_to_hex(private_key)} \n")
print(f"Public key (hex format): \n{serialize_public_key_to_hex(public_key)} \n")

Keys generated successfully. 

Private key (hex format): 
308204a20201000282010100b8ad61ea97de686ecf9ea7317cced9a53d6d49bc1caf9b5cf439e58fa5c725574c84872dce79b8d9ad17aca8ac466e470e91207185901f61efa9223dc46e49d19e76824da4f88b06b7b9d6eab0f741d71e63c7cafce8b21684b710cc33e6a45a2f17e4f7ce9fdf4a1841eadebf085caf7980ed869067cd10c996e69a7156c46fc49772cba6ab87c80c2dd4d99448c1675f08fb84c83b9911f85cea0efa5b27a27d46711ed96a58a33d8b8013d4ca90d59b5e338ef29bdde9eaa1823cbfe6dd5fd11eb256b8469ae773e16e0d2a8a4685096ad973226f75809dbc30cc7b5f69a5d23419013bb9d932c6760bb1a43ea17cf59f9dd0bae2ff86ab7bc080747aca3d02030100010282010029a2a346efb6cb7e220e4f7ce379c20d542839bdb08cf1beb6ebffa28f5a32c3b8034ff3a68f803a8920e3830295fd6245393862879d6ac9da6d9c03e76f82f280a290b57f1a2ec3d4b38fedc8f871ae555263169346c5f1b8398a8eb32bce456bc51c10b75389548bdbfe9ef9dc218349262546d773238323a3c2f93250c0c7dce5d0c9e314fc15a4e26a5422bc4694c43538febb98b236411b7a64a63d9cc51558b8f4253606eb08e87975839e1efc5090f5055fe68a1d04b0ff12a73bcf02e33e

In [37]:
# Block 2: Encryption 

# Encryption needs to be performed using public key

plaintext_message = input('Enter the message to encrypt: ')
pb_key_hex = input('Enter the public key (hex format) for encryption: ')

# Deserialize the public key from hex
public_key = deserialize_public_key_from_hex(pb_key_hex)

# Encrypt the message using the public key
encrypted_message = encrypt_message(plaintext_message, public_key)
print(f"\nEncrypted message: {encrypted_message} \n")


Enter the message to encrypt:  12345
Enter the public key (hex format) for encryption:  30820122300d06092a864886f70d01010105000382010f003082010a0282010100b8ad61ea97de686ecf9ea7317cced9a53d6d49bc1caf9b5cf439e58fa5c725574c84872dce79b8d9ad17aca8ac466e470e91207185901f61efa9223dc46e49d19e76824da4f88b06b7b9d6eab0f741d71e63c7cafce8b21684b710cc33e6a45a2f17e4f7ce9fdf4a1841eadebf085caf7980ed869067cd10c996e69a7156c46fc49772cba6ab87c80c2dd4d99448c1675f08fb84c83b9911f85cea0efa5b27a27d46711ed96a58a33d8b8013d4ca90d59b5e338ef29bdde9eaa1823cbfe6dd5fd11eb256b8469ae773e16e0d2a8a4685096ad973226f75809dbc30cc7b5f69a5d23419013bb9d932c6760bb1a43ea17cf59f9dd0bae2ff86ab7bc080747aca3d0203010001



Encrypted message: l09IlKyYVN8SiAYeRo+zWIiWSYBHEFrH1jxoZFqYEdg/xq0+yl+bEWiLffydxIlEY0Hk9gDdVDqBpS7m0+nRnAPxRfujomjd1OmMY+ggamU++6+5YF88Gy/2NvMbzu+PhzlfCX777Fweh00cF0FWBYmt1Ss6tZPZI1jI+BZ4v4Zjr2rohTAMxzjH6YE8dFBjPJZzlI/URvdc7FfTsZVOYXaIUurhHNRzB/ZougM3PsmkXGoZlbyTXqB5DbVnCk8nNHt6F+Uvqt6BVBfUjKgUFFu1rlvwfOdVhpM+BYSEmqnQZLJIDKQ2Pr21SNecQjzWgUs8Qt4FaK1I3JVsxy54aQ== 



In [41]:
# Block 3: Decryption

# Encryption needs to be performed using private key

encrypted_message = input('Enter the encrypted message (ciphertext): ')
pr_key_hex = input('Enter the private key (hex format) for decryption: ')

# Deserialize the private key from hex
private_key = deserialize_private_key_from_hex(pr_key_hex)

# Decrypt the message using the private key
decrypted_message = decrypt_message(encrypted_message, private_key)
print(f"\nDecrypted message: {decrypted_message} \n")


Enter the encrypted message (ciphertext):  l09IlKyYVN8SiAYeRo+zWIiWSYBHEFrH1jxoZFqYEdg/xq0+yl+bEWiLffydxIlEY0Hk9gDdVDqBpS7m0+nRnAPxRfujomjd1OmMY+ggamU++6+5YF88Gy/2NvMbzu+PhzlfCX777Fweh00cF0FWBYmt1Ss6tZPZI1jI+BZ4v4Zjr2rohTAMxzjH6YE8dFBjPJZzlI/URvdc7FfTsZVOYXaIUurhHNRzB/ZougM3PsmkXGoZlbyTXqB5DbVnCk8nNHt6F+Uvqt6BVBfUjKgUFFu1rlvwfOdVhpM+BYSEmqnQZLJIDKQ2Pr21SNecQjzWgUs8Qt4FaK1I3JVsxy54aQ==
Enter the private key (hex format) for decryption:  308204a20201000282010100b8ad61ea97de686ecf9ea7317cced9a53d6d49bc1caf9b5cf439e58fa5c725574c84872dce79b8d9ad17aca8ac466e470e91207185901f61efa9223dc46e49d19e76824da4f88b06b7b9d6eab0f741d71e63c7cafce8b21684b710cc33e6a45a2f17e4f7ce9fdf4a1841eadebf085caf7980ed869067cd10c996e69a7156c46fc49772cba6ab87c80c2dd4d99448c1675f08fb84c83b9911f85cea0efa5b27a27d46711ed96a58a33d8b8013d4ca90d59b5e338ef29bdde9eaa1823cbfe6dd5fd11eb256b8469ae773e16e0d2a8a4685096ad973226f75809dbc30cc7b5f69a5d23419013bb9d932c6760bb1a43ea17cf59f9dd0bae2ff86ab7bc080747aca3d02030100010282010029a2a3


Decrypted message: 12345 



## What is a Digital Signature? (Non-repudiation)
A digital signature is like signing a letter with your unique signature, but instead of using a pen, you use your private key to "sign" the message. When someone else gets your signed message, they can use your public key to check if the message is really from you and if it hasn't been tampered with.

Here’s the process:

Sign the Message: You "sign" the message using your private key.
Verify the Signature: Anyone can check the signature with your public key to make sure it's really you who sent the message and that it hasn’t been altered.

### Digital Signature (Signing the Message)
This is where you take a message and create a digital signature with your private key. It’s like putting your personal stamp on the message so others know it’s truly from you.

*signature = sign_message(message_to_sign, private_key)*

You create a signature using your private key. This signature is unique to both your message and your private key.
The signature is like a scrambled code that only your private key could create.
The message itself is not encrypted here, but it’s signed so that people can verify its authenticity.

#### Why Sign a Message?
- Proof of Identity: It proves the message came from you (the private key holder).
- Integrity: It ensures the message wasn’t altered during transmission.

### Signature Verification (Checking the Signature)
Now, if someone receives your signed message, they can check that it's really from you by verifying it with your public key.

In the code:

*is_valid = verify_signature(message_to_sign, signature_to_verify, public_key)*

Here’s what happens:

Verification: The public key is used to check the signature and confirm if it matches the message. If it does, the signature is valid.
If the message or signature doesn’t match, it means something is wrong (like someone tried to change the message), and the verification will fail.

Simple Example:
1. Alice writes a message and signs it with her private key.
2. Bob receives the message and Alice’s signature.
3. Bob uses Alice’s public key to verify that:
4. The message really came from Alice.
5. The message hasn’t been changed.


### What is Necessary for Signature Verification:
- Original Message: The message that was signed.
- Digital Signature: A unique signature created by the sender using their private key.
- Public Key: The public key of the sender used to verify the signature.
- Hashing Algorithm: A function that converts the message into a fixed-length string (e.g., SHA-256).
- Signing Algorithm: RSA (with padding) used to sign the hashed message.

Process:
1. Hash the Message: The recipient hashes the original message using the same algorithm (e.g., SHA-256).
2. Decrypt the Signature: The signature is decrypted using the sender’s public key to retrieve the original hash.
3. Compare Hashes: If the decrypted hash matches the hash of the message, the signature is valid. Otherwise, it’s invalid.


In [47]:
# Block 6. Hashing function

# Example usage
text = input('Enter the Student ID to get the HASH using SHA256: ')
hash_result = str(get_text_hash(text))
print(f"Hash of the text: {hash_result}")


Enter the Student ID to get the HASH using SHA256:  098765


Hash of the text: 4a9ca4596692e94f9d2912b06a0d007564a22ee750339a6021c2392149b25d6d


In [49]:
# Block 4: Digital Signature

message_to_sign = input('Enter the message to digitally sign: ')
pr_key_hex = input('Enter your private key (hex format) for signing: ')

# Deserialize the private key from hex
private_key = deserialize_private_key_from_hex(pr_key_hex)

# Sign the message using the private key
signature = sign_message(message_to_sign, private_key)
print(f"\nDigital signature: {signature} \n")



Enter the message to digitally sign:  4a9ca4596692e94f9d2912b06a0d007564a22ee750339a6021c2392149b25d6d
Enter your private key (hex format) for signing:  308204a40201000282010100c271fbcb8f015d61ffef4c7df16c8f0e5e42f946e282ab5ba614d43470d5a2b6b8aedb702ff0a8a09e32323fdb13df2d7f7002a53691c1d0df6b212cb93d2ecb94905484fbc890ab07e806c03d5cd817cb586e94ac2426a841d65aa39ed80972effae61f9236d704df3b909ef7b0620a52554558ce7e11641be8916416e9a7ef3c2b944e8d9b311d69319464695b6b7cb7af5db8ab1a96932105b38404c640499e5e86561fb8f76857c529425d7ff528d1841f4a67fe74bc16bccf23880354de55634d205790a9ef77dfa2613f963bfe262a2d5ff0872d0616ee5a8f5a2c4e1f7764f07ec5e5e0085ea9ecc1b0ae1d4d43ae3eea549b1fc6047e458facb0cb0d02030100010282010002c2dc51ff09cd90e356723bd753071b9956edfd21334f1c58d90e2860c462273486e447440f03072571172e4bf49f120f1d4a020abcfa6bf665f3444b1a89f55cd6126c0a71d85136940769a7de8f6f4b10b80e6bb3a76ee0aa2d3978047aeb03461b532e2924333c9524bdac03c1faefaf5f043a2e708b7f2b635c956cb6221e0328b13043a8ad4a18b299e78614cc942de


Digital signature: ecXkCyv3P0kid4EaDBfwZHVXbhBis2+uUqEPwHZHI596MxT/6Xpg0Q/sZqr4khfqu6KCwyPgslvFDUPU7Tksu/6aO67W66pe3hxG9ebuXEVG5wQmDVoP/j3jDa1SxXi9GXYU1wyz8bt/0+jOPmC0YUGXDM64OdZN3VtN1X1RxGZBVHuFxB/xsOc4SPhYede4NaF43tTbruW+tc5NZVxv2AUkCUbV8Jwlw90HkiDqrrTq1zeoL/H4iJWfULMVUhm6n/BFtZY+zLZsyxX20hYwleSsSaiMZv1AsLwIY/EBz7ZfbAT9VaKDiw7Yvz83/Ozi9OoT6BJs+OaG2IQHUllRNw== 



In [59]:
# Block 5: Signature Verification

signature_to_verify = input('Enter the signature to verify: ')
print('\n')
pb_key_hex = input('Enter the public key (hex format) for verification: ')
print('\n')
# Deserialize the public key from hex
public_key = deserialize_public_key_from_hex(pb_key_hex)

# Assuming 'message_to_sign' is already defined earlier or received as input
message_to_sign = input('Enter the student ID HASH to verify: ')
print('\n')
# Verify the signature using the public key
is_valid = verify_signature(message_to_sign, signature_to_verify, public_key)

if is_valid:
    print("\nThe signature is valid.\n")
    print(f"Original message: {message_to_sign}\n")
else:
    print("\nThe signature is invalid.\n")

Enter the signature to verify:  ecXkCyv3P0kid4EaDBfwZHVXbhBis2+uUqEPwHZHI596MxT/6Xpg0Q/sZqr4khfqu6KCwyPgslvFDUPU7Tksu/6aO67W66pe3hxG9ebuXEVG5wQmDVoP/j3jDa1SxXi9GXYU1wyz8bt/0+jOPmC0YUGXDM64OdZN3VtN1X1RxGZBVHuFxB/xsOc4SPhYede4NaF43tTbruW+tc5NZVxv2AUkCUbV8Jwlw90HkiDqrrTq1zeoL/H4iJWfULMVUhm6n/BFtZY+zLZsyxX20hYwleSsSaiMZv1AsLwIY/EBz7ZfbAT9VaKDiw7Yvz83/Ozi9OoT6BJs+OaG2IQHUllRNw==


Enter the public key (hex format) for verification:  30820122300d06092a864886f70d01010105000382010f003082010a0282010100c271fbcb8f015d61ffef4c7df16c8f0e5e42f946e282ab5ba614d43470d5a2b6b8aedb702ff0a8a09e32323fdb13df2d7f7002a53691c1d0df6b212cb93d2ecb94905484fbc890ab07e806c03d5cd817cb586e94ac2426a841d65aa39ed80972effae61f9236d704df3b909ef7b0620a52554558ce7e11641be8916416e9a7ef3c2b944e8d9b311d69319464695b6b7cb7af5db8ab1a96932105b38404c640499e5e86561fb8f76857c529425d7ff528d1841f4a67fe74bc16bccf23880354de55634d205790a9ef77dfa2613f963bfe262a2d5ff0872d0616ee5a8f5a2c4e1f7764f07ec5e5e0085ea9ecc1b0ae1d4d43ae3eea549b1fc6047e458facb0cb0d0203010001


Enter the student ID HASH to verify:  4a9ca4596692e94f9d2912b06a0d007564a22ee750339a6021c2392149b25d6d





The signature is valid.

Original message: 4a9ca4596692e94f9d2912b06a0d007564a22ee750339a6021c2392149b25d6d

